In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from statsmodels.graphics.gofplots import qqplot

df = pd.read_csv("data/TAQ_30Min_AAPL_2023_normalized.csv")
df.index = df.datetime
df["Log_Turnover"] = np.log(df["Normalized_TURNOVER"])
df.info()

## Preliminary: QQ plot
Since the Kalman filter paper has one

In [ ]:
qq_nolog = qqplot(df["Normalized_TURNOVER"], line="s")
plt.tight_layout()
plt.show()
plt.savefig("nolog_qqplot.pdf")

In [ ]:
qq_log = qqplot(np.log(df["Normalized_TURNOVER"]), line="s")
plt.tight_layout()
plt.show()
plt.savefig("log_qqplot.pdf")

## Define parameter object

In [ ]:
class Params:
    def __init__(self, pi, Sigma, a_eta, a_mu, sigma_eta_sq, sigma_mu_sq, r, phi):
        # pi and Sigma go into $x_t ~ \mathcal{N}(\pi_t, \Sigma_t)$
        self.pi = pi
        self.Sigma = Sigma
        # a_eta and a_mu define the state transition matrix A = [a_eta 0; 0 a_mu]
        self.a_eta = a_eta
        self.a_mu = a_mu
        # sigma_eta and sigma_mu define the covariance matrix Q = [sigma_eta^2 0; 0 sigma_mu^2]
        # for the Gaussian noise in the state transition w_t ~ \mathcal{N}(0, Q_t)
        self.sigma_eta_sq = sigma_eta_sq
        self.sigma_mu_sq = sigma_mu_sq
        # r goes into v_t ~ \mathcal{N}(0,r) where v_t is the noise in observation t
        self.r = r
        # phi is the seasonality parameter.
        # It's a vector in $\mathbb{R}^T$ where T is the number of intraday observations in a day
        self.phi = phi
        
    def A(self, tau):
        a1 = 1.0
        a2 = 1.0
        if tau % 13 == 0: # tau = kT for some integer K, T is the # of observations in a day
            a1 = self.a_eta
            a2 = self.a_mu
        return np.array([[a1, 0.0], [0.0, a2]])
    
    def Q(self, tau):
        a1 = 0.0
        a2 = 0.0
        if tau % 13 == 0: # tau = kT for some integer K, T is the # of observations in a day
            a1 = self.sigma_eta_sq
            a2 = self.sigma_mu_sq
        return np.array([[a1, 0.0], [0.0, a2]])

In [ ]:
C = np.ones((1,2))
C.shape

## Kalman filtering

In [ ]:
# initialization, I got this from the Kalman filtering paper
theta = Params(np.zeros(2), np.identity(2)*0.5, 1.0, 1.0, 0.0025, 0.0025, 0.0005, np.array([0.6, 0.25, 0.0, -0.15, -0.3, -0.45, -0.5, -0.6, -0.5, -0.25, -0.3, -0.1, 0.4]))

In [ ]:
def kalman_filtering(tau, x_hat_tau, y_tau_plus, Sigma_tau_tau, params):
    A = params.A(tau)
    x_hat_tau_plus = A @ x_hat_tau # predict mean
    Sigma_tau_plus = A @ Sigma_tau_tau @ A.T + params.Q(tau) # predict covariance
    
    # compute Kalman gain
    K_tau_plus = Sigma_tau_plus @ C.T @ np.linalg.inv(C @ Sigma_tau_plus @ C.T + params.r)
    
    # correct conditional mean
    x_hat_next = x_hat_tau_plus + K_tau_plus @ (y_tau_plus - params.phi[tau%13] - C@x_hat_tau_plus)
    Sigma_next = Sigma_tau_plus - K_tau_plus @ C @ Sigma_tau_plus
    #print("x_hat_next", x_hat_next.shape, "Sigma_next", Sigma_next.shape)
    return x_hat_next, Sigma_next

In [ ]:
# set up and run a dimensional test
y_1 = df.head(1)["Log_Turnover"]
x_1 = np.reshape(np.array([y_1/2, y_1/2]), 2)
Sigma_1 = np.eye(2)
x_plus, Sigma_plus = kalman_filtering(1, x_1, y_1, Sigma_1, theta)
print("Shape should be {}: x.shape = {}".format(2, x_plus.shape))
print("Shape should be {} x {}: Sigma.shape = {}".format(2, 2, Sigma_plus.shape))

## Kalman smoothing

In [ ]:
def kalman_smoothing(x_t, ys, Sigma_t, params):
    # this uses the outputs from the filtering algorithm
    # NOTE THAT x_t is a shorthand in the next few lines for x_{t|t} and Sigma_t := Sigma_{t|t}
    N = ys.shape[0]
    x_ts = []
    Sigma_ts = []
    
    # this is an unsightly way to code it but I think it makes more sense
    for t in range(0, N):
        x_t, Sigma_t = kalman_filtering(t, x_t, ys[t-1], Sigma_t, params)
        x_ts.append(x_t)
        Sigma_ts.append(Sigma_t)
        
    x_N, Sigma_N = x_ts[-1], Sigma_ts[-1]
    # Now we have x_{N|N}, Sigma_{N|N}
    
    x_tau_n = x_N # this is the initialization of x_{t+1|N} and Sigma_{t+1|N}
    Sigma_tau_n = Sigma_N
    Lt = np.zeros_like(Sigma_t)
    
    for t in range(N-1, 0, -1):
        A = params.A(t)
        # in here, Sigma_ts[t-1] is Sigma_{t|t} because of 0-indexing
        Sigma_tau_plus = A @ Sigma_ts[t-1] @ A.T + params.Q(t)
        x_hat_tau_plus = A @ x_ts[t-1]
        
        Lt = Sigma_ts[t-1] @ A.T @ np.linalg.inv(Sigma_tau_plus)
        x_tau_n = x_ts[t-1] + Lt @ (x_tau_n - x_hat_tau_plus)
        Sigma_tau_n = Sigma_ts[t-1] + Lt @ (Sigma_tau_n - Sigma_tau_plus) @ Lt.T
    return np.reshape(x_tau_n, (2,1)), Sigma_tau_n, Lt, x_ts, Sigma_ts # this is x_{t|N} and Sigma_{t|N}

In [ ]:
ys = df["Log_Turnover"].to_numpy()
#ys = np.reshape(ys, (int(ys.shape[0]/T), T)) # reshape ys to (N_days, T)

N_train = 50

x_tau_n, Sigma_tau_n, _, _, _ = kalman_smoothing(x_1, ys[0:N_train], Sigma_1, theta)
# dimensional check again
print(x_tau_n.shape)
print(Sigma_tau_n.shape)

In [ ]:
def compare(params1, params2):
    err = np.mean(np.abs(params1.pi - params2.pi)) + np.mean(np.abs(params1.Sigma - params2.Sigma)) + \
          np.abs(params1.a_eta - params2.a_eta) + np.abs(params1.a_mu - params2.a_mu) + \
          np.abs(params1.sigma_eta_sq - params2.sigma_eta_sq) + \
          np.abs(params1.sigma_mu_sq - params2.sigma_mu_sq) + \
          np.abs(params1.r - params2.r) + np.mean(np.abs(params1.phi - params2.phi))
    return err/8.0 # since there are 8 terms
        

## Expectation maximization
In this step we want to predict $x_\tau = [\eta_\tau\ \mu_\tau]^\top \in \mathbb{R}^2$ which is the hidden state vector. The variables $\eta_\tau$ and $\mu_\tau$ are the daily average and intraday dynamic part of the log volume.

In [ ]:
def sum_matrices(Ps):
    result = np.zeros_like(Ps[0])
    for Pi in Ps:
        result += Pi
    return result

I = 13

def em(x_1, ys, params, maxsteps=10, tol=1e-1):
    step = 0; err = np.Inf
    N = ys.shape[0]
    N_days = int(N/I)
    print("N = {}, N_days={}".format(N, N_days))
    
    while step < maxsteps and err > tol:
        
        Ps = [] # this is an array of P_t in REVERSE order
        P_minuses = [] # this is an array of P_{t|t-1} in REVERSE order, e.g. P_{N|N-1} is first and P_{1|0} is last
        xs = []
        x_tau = x_1
        Sigma_tau = params.Sigma
        Sigma_tau_minus_N = Sigma_tau # IDK how to initialize this
        
        for tau in range(N, 1, -1): # this is [N, N-1,...,2] because otherwise the indexing doesn't make sense for kalman smoothing
            # this line is x_{tau|N}, Sigma_{tau|N}, L_tau and x_ts = list of x_{tau|tau}, Sigma_ts = list of Sigma_{tau|tau}
            x_tau_n, Sigma_tau_n, L_tau, x_ts, Sigma_ts = kalman_smoothing(x_1, ys[0:tau], params.Sigma, params)
            x_tau = x_tau_n # line 5
            P_tau = Sigma_tau_n + x_tau_n @ x_tau_n.T # line 6
            
            # this line gets x_{tau-1|N}, Sigma-{tau-1|N} and L_{tau-1}
            x_tau_minus_n, Sigma_tau_minus_N, L_tau_minus, _, _ = kalman_smoothing(x_1, ys[0:tau-1], params.Sigma, params)
            # This is line 7 of Algorithm 3 which computes Sigma_{tau, tau-1|N}
            # Note that Sigma_{tau, tau-1|N} has a dependency on Sigma_{tau+1, tau|N} which makes sense except
            # we do not know what to initialize it to
            Sigma_tau_minus_N = Sigma_ts[tau-1] @ L_tau_minus.T + \
                                L_tau @ (Sigma_tau_minus_N - params.A(tau) @ Sigma_ts[tau-1]) @ L_tau_minus.T
            P_tau_minus = Sigma_tau_minus_N + x_tau_n @ x_tau_minus_n.T # line 8
            
            xs.append(x_tau)
            Ps.append(P_tau)
            P_minuses.append(P_tau_minus)
        Ps.reverse()
        P_minuses.reverse()
        print("{} first step done {}".format(step, len(Ps)))
        
        N = N - 1
        # now we have a list of P_t and P_{t|t-1}
        pi = x_tau # equation 17
        Sigma = Ps[-1] - x_tau @ x_tau.T # equation 18
        print("\tpi, sigma = {}, {}".format(pi, Sigma))
        P_sum = sum_matrices([Ps[I*i + 1] for i in range(N_days)]) # for equation 19
        P_minus_sum = sum_matrices([P_minuses[I*i + 1] for i in range(N_days)]) # for equation 19
        a_eta = P_minus_sum[0,0] / P_sum[0,0]
        
        P_sum2 = sum_matrices([Ps[i] for i in range(2, N)]) # for equation 20
        P_minus_sum2 = sum_matrices([P_minuses[i] for i in range(1, N-1)]) # for equation 20
        a_mu = P_minus_sum2[1,1] / P_sum2[1,1]
        
        sigma_eta_sq = 0.0 # equation 21
        print("\ta_eta = {}, a_mu = {}".format(a_eta, a_mu))
        for i in range(N_days):
            t = i*I+1
            sigma_eta_sq += (Ps[t] + a_eta**2.0 * Ps[t-1] - 2.0 * a_eta * P_minuses[t-1])[0,0]
        sigma_eta_sq /= (N_days - 1) # 
        
        sigma_mu_sq = 0.0 # equation 22
        for t in range(2, N):
            sigma_mu_sq = (Ps[t] + a_mu**2.0 * Ps[t-1] - 2.0 * a_mu * P_minuses[t-1])[1,1]
        sigma_mu_sq *= 1.0/(N - 1.0)
        print("\tsigma_eta_sq = {}, sigma_mu_sq = {}".format(sigma_eta_sq, sigma_mu_sq))
        r = 0.0
        for t in range(N): # equation 23, probably
            r += ys[t]**2 + np.sum(Ps[t]) - 2.0*ys[t] * np.sum(xs[t]) + \
                (params.phi[t%I])**2.0 - 2.0 * ys[t] * params.phi[t%I] + \
                2.0*params.phi[t%I] * np.sum(xs[t])
        r /= N
        
        phi = np.zeros(13)
        for i in range(N):
            phi[i%I] += (ys[i:i+1] - C@xs[i])[0,0]
        phi /= N_days
        print("\tr = {}, phi = {}".format(r, phi))
        print("{} second step done".format(step))
        
        
        params1 = Params(pi, Sigma, a_eta, a_mu, sigma_eta_sq, sigma_mu_sq, r, phi)
        err = compare(params, params1)
        params = params1
        
        print("{}, Error={}".format(step, err))
        step += 1
        N += 1
        
    return params
            

## Class interface
This does not do exactly what is requested but it is close.

In [ ]:
class KalmanPredictor:
    def __init__(self, params:Params, I=13):
        self.params = params
        self.I = I
    
    def x1(self):
        return multivariate_normal(self.params.pi.flatten(), self.params.Sigma).rvs(1)
    
    def train(self, log_returns, maxsteps=25, tol=0.01):
        x_1 = self.x1()
        self.params = em(x_1, log_returns, self.params, maxsteps=maxsteps, tol=tol)
    
    def predict(self, y_observed):
        # This will return a vector of y_hat that is the same size as y_observed
        # The assumption is we start at a hidden state x1 described by the distribution in self.params
        N = y_observed.size
        predictions = np.zeros_like(y_observed)
        x_t = self.x1()
        Sigma_t = self.params.Sigma
        predictions[0] = (C@x_t)[0]
        for i in range(1,N):
            x_t, Sigma_t = kalman_filtering(i, x_t, y_t, Sigma_t, self.params)
            predictions[i] = (C@x_plus)[0]
        return predictions

In [ ]:
new_params = em(x_1, ys[1:N_train*13], theta, maxsteps=25, tol=0.01)

In [ ]:
# TEST - undo this
# It seems like the phi parameter converges to a wrong thing. If we use the guess of phi it works much better
# WHY?
test_params = Params(new_params.pi, new_params.Sigma, new_params.a_eta, new_params.a_mu,
                     new_params.sigma_eta_sq, new_params.sigma_mu_sq, new_params.r, theta.phi)

In [ ]:
predictor = KalmanPredictor(test_params) # we already trained them in the previous cell, holdover from previous code

## Test Kalman filter with given params

In [ ]:
N = df["Log_Turnover"].size

In [ ]:
y_t = df.iloc[0:1]["Log_Turnover"]
x_t = np.reshape(np.array([y_t/2, y_t/2]), 2)
xs = [x_t,]

Sigma_t = np.identity(2)
sigmas = [Sigma_t,]

y_pred = [y_t]
y_pred = predictor.predict(df["Log_Turnover"])

In [ ]:
errs = [np.mean(np.square(y_pred[i] - df.iloc[i:i+1]["Log_Turnover"])) for i in range(N)]

In [ ]:
y_t = df.iloc[0:1]["Log_Turnover"]
x_t = np.reshape(np.array([y_t/2, y_t/2]), 2)

Sigma_t = np.identity(2)

y_pred_new = [y_t]

for i in range(N):
    y_t = df.iloc[i:i+1]["Log_Turnover"]
    x_plus, Sigma_plus = kalman_filtering(i, x_t, y_t, Sigma_t, test_params)
    y_pred_new.append((C@x_plus)[0])

In [ ]:
errs_new = [np.mean(np.square(y_pred_new[i] - df.iloc[i:i+1]["Log_Turnover"])) for i in range(N)]

### Comparison: hourly average over the year

In [ ]:
log_avgs = np.log(df["Hourly averages"])
errs_avg = [np.mean(np.square(log_avgs - df.iloc[i:i+1]["Log_Turnover"])) for i in range(N)]

In [ ]:
#plt.semilogy(np.mean(np.reshape(errs, (int(len(errs)/I), I)), axis=1), label="Average daily prediction error")
plt.xlabel("Day of year"); plt.ylabel("MSE error")
plt.semilogy(np.mean(np.reshape(errs_new, (int(len(errs_new)/I), I)), axis=1), label="Daily prediction error (Kalman)")
plt.semilogy(np.mean(np.reshape(errs_avg, (int(len(errs_avg)/I), I)), axis=1), linestyle="--", label="Daily prediction error (average)")

plt.legend()
plt.savefig("kalman_errors_year.pdf")
print("Average error: {}".format(np.mean(errs_avg)))
print("Kalman error: {}".format(np.mean(errs_new)))

In [ ]:
fig, axs = plt.subplots(5, sharex=True, figsize=(5,7))

axs[0].semilogy(np.exp(y_pred_new[-I:]),label="Predicted turnover (Kalman)", linestyle="-.", color="cornflowerblue")
axs[0].plot(np.exp(df.iloc[-I:]["Log_Turnover"].to_numpy()), label="True normalized turnover", color="black")
axs[0].plot(df.iloc[-I:]["Hourly averages"].to_numpy(), label="Predicted turnover (mean)", color="orange", linestyle="--")

for i in range(1,5):
    axs[i].semilogy(np.exp(y_pred_new[-I*(i+1):-I*i]), linestyle="-.", color="cornflowerblue")
    axs[i].plot(np.exp(df.iloc[-(i+1)*I:-i*I]["Log_Turnover"].to_numpy()), color="black")
    axs[i].plot(df.iloc[-(i+1)*I:-i*I]["Hourly averages"].to_numpy(), color="orange", linestyle="--")

daily_labels = ["9:30", "10:30", "11:30", "12:30", "13:30", "14:30", "15:30",]
plt.xticks(ticks=range(0, I, 2), labels=daily_labels)
fig.legend(loc="upper center"); plt.ylabel("Normalized turnover")
fig.tight_layout()
fig.savefig("kalman_prediction.pdf")